# NFLX Exercise - Analysis

Hi!

The intention of this notebook code is to answear questions about a Netflix database to our client.  I'll separate the notebook cells to make more easier to identify what question will be answeared now; also, I'll explain the logic in the code.
Questions:
 1. In the ‘Films (English)’ category, which film has the most appearances in our data set
 (NFLX Top 10 tab of the Sheet)? What were its average weekly hours viewed?
 2. In the 'Films (English)' category, which film has the lowest IMDb rating? What were its
 average weekly hours viewed?
 3. In the 'Films (Non-English)' category, which film has spent the most weeks in the top 10?
 To estimate the number of users who watched this film, what assumptions would you
 make andwhat risks are involved?
 ○ Pleaselimit your response to 150 words or less.
 4. Whatarethe risks of ignoring the data from the week of May 22nd when calculating the
 metrics from the previous questions?
 ○ Pleaselimit your response to 150 words or less.
 5. While we've indicated that the 'weekly_hours_viewed' data for the week of May 22nd
 cannot be used in our estimates, we may want to fill in this missing information for other
 analyses. As a Data Specialist, what methodology would you propose to estimate the
 'weekly_hours_viewed' for this missing week?

In [ ]:
#first of all, we should import pandas to read the dataset properly.
import pandas as pd
df = pd.read_excel('/content/Data Specialist NFLX Data.xlsx', sheet_name="NFLX Top 10")

In [ ]:
#as explained in the text, we have incomplete data from the week of may 2022. I will load another dataset that excludes this week.
filtered_df = df[df['week'] != '2022-05-22']

In [ ]:
#now we can go to the 1st question. for that, I'll filter the films that belongs to the english category and after that count its appearances in the dataset. with its appearances, I can make an average using the previously count.
english_films = filtered_df[filtered_df['category'] == 'Films (English)']

#we can simply use value_counts to count the appearances of show title, and after that use idxmax to return the index of the counting method.
count_films = english_films['show_title'].value_counts()
count_films_title = english_films['show_title'].value_counts().idxmax()
count_films_max = english_films['show_title'].value_counts().max()
print('Film with most weekly appearances: ',count_films_title)

#now we know that 'Sonic the Hedgehog' is the film with most appearances. to know the weekly hours watched avg, we can sum the weekly hours viewed and divide by the weekly appearances.
hours_viewed_titles = english_films.groupby('show_title')['weekly_hours_viewed'].sum()
hours_viewed_max_count = hours_viewed_titles[count_films_title]
avg_hours_viewed = hours_viewed_max_count/count_films_max
print('Avearage of watched hours: ',avg_hours_viewed)

Film with most weekly appearances:  Sonic the Hedgehog
Avearage of watched hours:  8550000.0


In [ ]:
#to answear the 2nd question, we have to make a join between 'NFLX Top 10' sheet and 'IMDB Rating' sheet. the key that will be used is 'show_title' and 'title'. in this case, we can bring the rating column to previously used df.
imdb_df = pd.read_excel('/content/Data Specialist NFLX Data.xlsx', sheet_name="IMDB Rating")
english_films_merge = english_films.merge(imdb_df[['title', 'rating']],
                                    how='left',
                                    left_on='show_title',
                                    right_on='title')

#we can drop the 'title' column from imdb rating, because we already have this key in the left dataset.
english_films_merge.drop(columns=['title'], inplace=True)

#before find out the lowest rating, we have to exclude the rating 0 in our dataset. according to the note, if a rating is unavailable on IMDb, we input it as 0 and exclude it from any ratings analyses.
english_films_merge_rating = english_films_merge[english_films_merge['rating'] != 0]

#now we can find out the lowest rating film on imdb. we will use the min function.
min_rating = english_films_merge_rating['rating'].min()
min_rating_title = english_films_merge_rating[english_films_merge_rating['rating'] == min_rating]['show_title'].values[0]
print('Film with lowest IMDB rating: ',min_rating_title)

#now we know that '365 Days: This Day' is the film with the lowest IMDB rating. to know the weekly hours watched avg, we can sum the weekly hours viewed and divide by the weekly appearances. we can use the same sum variable.
hours_viewed_min_sum = hours_viewed_titles[min_rating_title]
count_films_lowest = english_films['show_title'].value_counts()[min_rating_title]
avg_hours_viewed_lowest_rating = hours_viewed_min_sum/count_films_lowest
print('Avearage of watched hours: ',avg_hours_viewed_lowest_rating)

Film with lowest IMDB rating:  365 Days: This Day
Avearage of watched hours:  38696666.666666664


In [ ]:
#in the 3rd question, we need change the filter of the dataset to non english filmes and return the max value of "cumulative_weeks_in_top_10" column.
non_english_films = filtered_df[filtered_df['category'] == 'Films (Non-English)']

#returning the max value of "cumulative_weeks_in_top_10" and its title.
non_english_max_appearances = non_english_films['cumulative_weeks_in_top_10'].max()
title_non_english_max_appearances = non_english_films[non_english_films['cumulative_weeks_in_top_10'] == non_english_max_appearances]['show_title'].values[0]
print('Non english film with most weekly appearances: ',title_non_english_max_appearances)

#sum of weekly hours views in 'Through My Window' film
hours_viewed_tmw = df[df['show_title'] == title_non_english_max_appearances]['weekly_hours_viewed'].sum()
print('Total watched hours of Through my Window: ',hours_viewed_tmw)

#runtime of Through my Window
runtime_df = pd.read_excel('/content/Data Specialist NFLX Data.xlsx', sheet_name="Runtime")
runtime_tmw = (runtime_df[runtime_df['title']==title_non_english_max_appearances]['runtime'].values[0])/60
print('Runtime of Through my Window in hours:', runtime_tmw)

#if we divide the sum of total hours that users spent watching this film with the runtime of the film, we can estimate the number of people that watched the movie
people_watched_tmw = hours_viewed_tmw/runtime_tmw
print('Estimated number of people that watched the movie:', people_watched_tmw)

Non english film with most weekly appearances:  Through My Window
Total watched hours of Through my Window:  10380000
Runtime of Through my Window in hours: 1.9333333333333333
Estimated number of people that watched the movie: 5368965.517241379
